In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [2]:
!pip install evaluate datasets -q
!pip install accelerate -U -q
!pip install transformers[torch] -q

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

dataset = load_dataset("glue", "mrpc")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,No log,0.404728


TrainOutput(global_step=459, training_loss=0.5711429238579112, metrics={'train_runtime': 95.3722, 'train_samples_per_second': 38.46, 'train_steps_per_second': 4.813, 'total_flos': 965091351060480.0, 'train_loss': 0.5711429238579112, 'epoch': 1.0})

In [14]:
# 모델 저장
model_path = "paircl_bert_model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('paircl_bert_model/tokenizer_config.json',
 'paircl_bert_model/special_tokens_map.json',
 'paircl_bert_model/vocab.txt',
 'paircl_bert_model/added_tokens.json')

In [16]:
!ls

logs  paircl_bert_model  results  sample_data


In [15]:
!rm -rf ./paircl_bert__model

In [8]:
!ls ./paircl_bert_model

config.json  model.safetensors	training_args.bin


In [9]:
# mkdir과 함께 -p 옵션을 사용하여 상위 디렉터리를 만들 수 있습니다
!mkdir "/gdrive/MyDrive/kdt_231026/m7_nlp응용/data/paircl_bert_model"

In [17]:
!cp -r ./paircl_bert_model /gdrive/MyDrive/kdt_231026/m7_nlp응용/data/paircl_bert_model